In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.tokenize import word_tokenize
from time import time
import pickle

import os

In [2]:
df = pd.read_csv('train.csv')
poi_street_df = df["POI/street"].str.split("/", n = 1, expand = True) 
df["POI"]= poi_street_df[0] 
df["street"]= poi_street_df[1] 
df1 = df.drop(["POI/street"], axis=1) 
df1

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung
3,3,"toko dita, kertosono",toko dita,
4,4,jl. orde baru,,jl. orde baru
...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,,
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,


### Define function

In [3]:
def find_first_index(ra_split, st_split):
    
    if len(st_split) <= len(ra_split): # new
    
        num_iter = len(ra_split) - len(st_split) + 1
        overlap_list = []

        for i in range(num_iter):
            window = ra_split[i: i+len(st_split)]
            overlap = list(set(window) & set(st_split))
            overlap_list.append(len(overlap))

        max_overlap = [e for e in range(len(overlap_list)) if overlap_list[e] == max(overlap_list)]
        if len(max_overlap) == 1:
            return max_overlap[0]

        else:
            count_list = []
            for idx in max_overlap:
                subset_ra = ra_split[idx: idx+len(st_split)]
                count = 0
                for e in range(len(subset_ra)):
                    if subset_ra[e] not in st_split[e]:
                        count += 0
                    else:
                        count += 1
                count_list.append(count)
            index = count_list.index(max(count_list))
            return max_overlap[index]
    else:
        return 0

## Data Cleaning

### 1. Let's investigate discrepancies between `street` and `raw_address` labels.

In [4]:
df1.head(10)

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",,
2,2,setu siung 119 rt 5 1 13880 cipayung,,siung
3,3,"toko dita, kertosono",toko dita,
4,4,jl. orde baru,,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids,raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",,kem mel raya
7,7,tela keuramat kuta alam,,tela
8,8,gg. i wates magersari,,gg. i
9,9,bunga ncole ix 2,,bunga ncole ix


In [ ]:
start = time()

before_list1 = []
after_list1 = []

for row, col in df1.iterrows():
    
    raw_add = col['raw_address']
    
    # If a street name is extracted...
    if col['street'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_street = col['street']
        extr_street_split = word_tokenize(extr_street)
        
        # If the extracted street is in the raw address as an entire string, good!
        if extr_street in raw_add:
            df1.loc[row, 'cleaned_raw_address'] = raw_add
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_street_split)
            before = raw_add_split[index_in_ra: index_in_ra+len(extr_street_split)]
            before_list1.append(' '.join(before))
            
            before = extr_street_split
            after_list1.append(extr_street)
            
            updated_raw_add = ' '.join(raw_add_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?')          
            df1.loc[row, 'cleaned_raw_address'] = updated_raw_add
      
    # If a street name is originally an empty string, we just assume there's no error. 
    else:
        df1.loc[row, 'cleaned_raw_address'] = raw_add

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))
    
# Sanity checks
df1.head(10)

### 2. We also build a dictionary mapping the incorrect/incomplete street names in `raw_address` to the correct `street` labels. 

In [ ]:
street_mapping_dict = dict()

for key in before_list1: 
    for value in after_list1: 
        street_mapping_dict[key] = value 
        after_list1.remove(value) 
        break
        
print("Length of street mapping dictionary:", len(street_mapping_dict))

### 3. Now, let's rectify discrepancies between `POI` and `raw_address` labels.

In [ ]:
start = time()

before_list2 = []
after_list2 = []

for row, col in df1.iterrows():
    
    raw_add = col['cleaned_raw_address']
    
    # If a POI name is extracted...
    if col['POI'] != "":

        raw_add_split = word_tokenize(raw_add)
        
        extr_poi = col['POI']
        extr_poi_split = word_tokenize(extr_poi)
        
        # If the extracted POI is in the raw address as an entire string, good!
        if extr_poi in raw_add:
            df1.loc[row, 'cleaned_raw_address_1'] = raw_add
        
        # This is where there are discrepancies!
        else:
            index_in_ra = find_first_index(raw_add_split, extr_poi_split)
            before = raw_add_split[index_in_ra: index_in_ra+len(extr_poi_split)] 
            before_list2.append(' '.join(before))
            
            before = extr_poi_split
            after_list2.append(extr_poi)
            
            updated_raw_add = ' '.join(raw_add_split).replace(' ,', ',').replace(' .', '.').replace(' )', ')').replace(' (', '(').replace(' ?', '?')
            df1.loc[row, 'cleaned_raw_address_1'] = updated_raw_add
      
    # If a POI name is originally an empty string, we just assume there's no error. 
    else:
        df1.loc[row, 'cleaned_raw_address_1'] = raw_add

print("Executed in {} minutes.".format(round((time() - start)/60, 3)))
    
# Sanity checks
df1.head(10)

In [ ]:
poi_mapping_dict = dict()

for key in before_list2: 
    for value in after_list2: 
        poi_mapping_dict[key] = value 
        after_list2.remove(value) 
        break
        
print("Length of POI mapping dictionary:", len(poi_mapping_dict))

### 4. Now, we've cleaned the raw address to make sure that it tallies with both the extracted `street` and `POI` labels. Finally, let's tidy things up. 

In [ ]:
# Select required columns
cleaned_df = df1[['id','cleaned_raw_address_1', 'POI', 'street']]

# Rename columns
cleaned_df.columns = ['id', 'raw_address', 'POI', 'street']

# Preview
cleaned_df.head()

### 5. Now that we have the two mapping dictionaries, let's check whether some of those truncated words exist in the test data, and if so, replace them with the correct labels. 

In [ ]:
test_df = pd.read_csv('/kaggle/input/scl-2021-ds/test.csv')
test_df

It might be risky to replace single words like `par` in the test data as they may form part of a bigger word that is different from the intended word. For example, we should not replace "par" in the address "daya paru 43" with its value in the `street_mapping_dict`, "parigi". This is less likely the case if there are two or more words.

Let's try to filter out single words from the list.

In [ ]:
new_street_mapping_dict = {k: v for k, v in street_mapping_dict.items() if len(k.split()) > 1}
new_poi_mapping_dict = {k: v for k, v in poi_mapping_dict.items() if len(k.split()) > 1}

print("Length of street mapping dictionary after removing single words:", len(new_street_mapping_dict))
print("Length of POI mapping dictionary after removing single words:", len(new_poi_mapping_dict))

In [ ]:
# Replace truncated words in raw_address of test set with correct street labels
for row, col in test_df.iterrows():
    for k, v in new_street_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)

# Replace truncated words in raw_address of test set with correct POI labels
for row, col in test_df.iterrows():
    for k, v in new_poi_mapping_dict.items():
        if k in col['raw_address']:
            test_df.loc[row, 'raw_address'] = test_df.loc[row, 'raw_address'].replace(k, v)

test_df

### 6. Save the cleaned-up train and test datasets, as well as the two mapping dictionaries.

In [ ]:
# Save the cleaned train dataset
cleaned_df.to_csv('cleaned_train.csv', index=False)

# Save the cleaned test dataset
test_df.to_csv('cleaned_test.csv', index=False)

In [ ]:
# Save the dictionaries
s_file = open("street_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, s_file)
s_file.close()

p_file = open("poi_mapping_dict.pkl", "wb")
pickle.dump(street_mapping_dict, p_file)
p_file.close()

### Note: Run the following codes to load the two dictionaries

In [ ]:
# s_file = open("street_mapping_dict.pkl", "rb")
# street_mapping_dict = pickle.load(s_file)
# print(street_mapping_dict)

# p_file = open("poi_mapping_dict.pkl", "rb")
# poi_mapping_dict = pickle.load(p_file)
# print(poi_mapping_dict)